In [1]:
%matplotlib inline

import requests
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from time import sleep
import seaborn as sns
sns.set()

def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [2]:
data_path = r'./data/cdcpniweekly/'
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [3]:
years = range(2016, 2021)
weeks = range(1, 54)

- Any given week's report will have all weeks up until that week.
- It appears that the at least some of the early reports had a two week lag, not a one week lag.
- A flu season starts on Week 40 of one year and goes to week 39 of the next year.
Example:
- The first week of the 2017-2018 flu year is https://www.cdc.gov/flu/weekly/weeklyarchives2017-2018/data/nchsdata40.csv
- The Last week of the 2017-2018 flu year is https://www.cdc.gov/flu/weekly/weeklyarchives2017-2018/data/nchsdata39.csv


Uncomment the next cell to pull the data down to your local machine.

# Please don't repeatedly pull down the data as it creates unnecessary work for the CDC's servers. I have included the result of running this code in the repo. So unless you want to grab more recent snapshots down the road there's no point in running it. Just run the notebook as is and it will use the data in this repo.

In [4]:
# # CDC DATA Format
# # https://www.cdc.gov/flu/weekly/weeklyarchives2017-2018/data/nchsdata42.csv
# for year in years:
#     for week in weeks:
#         if year == 2020 and week > 13:
#             next
#         else: 
#             flu_year = str(year - 1) + '-' + str(year)
#             request_url = ('https://www.cdc.gov/flu/weekly/weeklyarchives' +
#                            flu_year + '/data/nchsdata' + str(week).zfill(2) + '.csv')
#             print(request_url)
#             request = requests.get(request_url)
#             isStatusOK = request.status_code == requests.codes.ok
#             if isStatusOK:
#                 decoded_content = request.content.decode('utf-8')
#                 with open('./data/cdcpniweekly/' + flu_year + '-' + str(week).zfill(2) + '.csv', 'w') as my_data_file:
#                     print('isOK')
#                     my_data_file.write(decoded_content)

In [5]:
def calendarWeektoFluSeasonWeek(week):
    intWeek = int(week)
    if intWeek > 39:
        return intWeek - 39
    else:
        return intWeek + 13
    
def FluSeasonWeektoCalendarWeek(week):
    intWeek = int(week)
    if intWeek < 14:
        return intWeek + 39
    else:
        return intWeek - 13
    
weekly_reports = []
path = data_path # use your path
all_files = glob.glob(path + "/*.csv")
for filename in all_files:
#     print(filename)
    if 'combined' not in filename:
        flu_season_start_year = filename[20:24]
        flu_season_end_year = filename[25:29]
        calendar_week = filename[30:32]
        flu_season_week = calendarWeektoFluSeasonWeek(calendar_week)
#         print(flu_year_start, flu_year_end, calendar_week)
        df = pd.read_csv(filename, index_col=None, header=0)
        df['calendarWeek'] = int(calendar_week)
        df['filename'] = filename
        df['fluSeason'] = filename[20:29]
        df['fluSeasonStartYear'] = flu_season_start_year
        df['fluSeasonEndYear'] = flu_season_end_year
        df['fluSeasonWeek'] = flu_season_week
        df['reportId'] = df['fluSeason'] + '-' + df['fluSeasonWeek'].apply(str).str.zfill(2)
#         df['snapshotYear'] = snapshotYear
#         df['snapshotCalendarWeek'] = week
#         df['snapshot'] = df['fluSeason'] + df['snapshotCalendarWeek']
        weekly_reports.append(df)
combined_data = pd.concat(weekly_reports, ignore_index=True)
combined_data.to_csv(data_path + 'combined.csv')

In [6]:
# display_all(combined_data.query('calendarWeek == 39'))

In [7]:
report_ids = list(np.sort(combined_data['reportId'].unique()))
report_ids[:3]

['2015-2016-01', '2015-2016-02', '2015-2016-03']

In [8]:
reports = {}
for report_id in report_ids:
    pivoted_data = combined_data.query('reportId == @report_id').pivot_table(
        values='Pneumonia Deaths',
        columns='Year',
        index='Week')
    reports[report_id] = pivoted_data
#     snapshots[snapshot] = pd.concat([pivoted_data[40:], pivoted_data[:39]])
#     snapshots[snapshot]['fluWeek'] = list(range(1, len(snapshots[snapshot]) + 1))
#     snapshots[snapshot] = snapshots[snapshot].set_index('fluWeek')

In [9]:
reports["2015-2016-01"]

Year,2010,2011,2012,2013,2014,2015
Week,,,,,,
1,NaN,4817.0,4360.0,5620.0,4555.0,5467.0
2,NaN,4725.0,4262.0,6080.0,5047.0,5477.0
3,NaN,4715.0,4274.0,5941.0,5021.0,5106.0
4,NaN,4820.0,4232.0,5649.0,4789.0,4772.0
5,NaN,4705.0,4029.0,5291.0,4629.0,4641.0
6,NaN,4755.0,4027.0,5028.0,4480.0,4456.0
7,NaN,4970.0,4133.0,4889.0,4390.0,4340.0
8,NaN,5013.0,4195.0,4581.0,3995.0,4377.0
9,NaN,4822.0,4238.0,4489.0,3873.0,4332.0


In [10]:
[""]*12

['', '', '', '', '', '', '', '', '', '', '', '']

In [11]:
# CDC DATA Format
dash_styles = [""]*15

colors = sns.color_palette("husl", 12)

# pallete = {
#     "2008 - 2009": colors[0],
#     "2009 - 2010": colors[1],
#     "2010 - 2011": colors[2],
#     "2011 - 2012": colors[3],
#     "2012 - 2013": colors[4],
#     "2013 - 2014": colors[5],
#     "2014 - 2015": colors[6],
#     "2015 - 2016": colors[7],
#     "2016 - 2017": colors[8],
#     "2017 - 2018": colors[9],
#     "2018 - 2019": colors[10],
#     "2019 - 2020": colors[11],
# }

pallete = {
    2009: colors[0],
    2010: colors[1],
    2011: colors[2],
    2012: colors[3],
    2013: colors[4],
    2014: colors[5],
    2015: colors[6],
    2016: colors[7],
    2017: colors[8],
    2018: colors[9],
    2019: colors[10],
    2020: colors[11],
}

sns.set_style("dark")
sns.set_context("notebook", font_scale=2.5, rc={"lines.linewidth": 5})
def plot_data(x):
    week = x[-2:]
    start_year = x[0:4]
    end_year = x[5:9]
    if int(week) < 14:
        year = start_year
        week = int(week) + 39
    else:
        year = end_year
        week = int(week) - 13
    plt = sns.relplot(kind="line",
                      data=reports[x],
                      dashes=dash_styles,
                      height=9,
                      aspect=1.5,
                      hue="Year",
                      palette=pallete)
    plt.set(ylim=(0,7000))
    plt.set(xlim=(0,53))
    plt.set(title="U.S. Provisional Pnuemonia Deaths as of " + year + ' week ' + str(week))
    plt.set(xlabel='Week of Year', ylabel='Death Count')

w = widgets.Dropdown(options=report_ids, value=report_ids[0])
interact(plot_data, x=w)

interactive(children=(Dropdown(description='x', options=('2015-2016-01', '2015-2016-02', '2015-2016-03', '2015…

<function __main__.plot_data(x)>

This look is what creates the animation

In [12]:
for value in report_ids:
    w.value = value